# Configuring and Using Dataset Features 

This notebook shows model developers how they can learn more about configuring and using the features in `init_dataset` These instructions include the code required to:

- ...
- ...

## ValidMind at a glance

ValidMind's platform enables organizations to identify, document, and manage model risks for all types of models, including AI/ML models, LLMs, and statistical models. As a model developer, you use the ValidMind Developer Framework to automate documentation and validation tests, and then use the ValidMind AI Risk Platform UI to collaborate on documentation projects. Together, these products simplify model risk management, facilitate compliance with regulations and institutional standards, and enhance collaboration between yourself and model validators.

If this is your first time trying out ValidMind, we recommend going through the following resources first:

- [Get started](https://docs.validmind.ai/guide/get-started.html) — The basics, including key concepts, and how our products work
- [Get started with the ValidMind Developer Framework](https://docs.validmind.ai/guide/get-started-developer-framework.html) —  The path for developers, more code samples, and our developer reference

## Before you begin

::: {.callout-tip}
### New to ValidMind? 
For access to all features available in this notebook, create a free ValidMind account. 

Signing up is FREE — [**Sign up now**](https://app.prod.validmind.ai)
:::


This notebook requires an OpenAI API secret key to run. If you don't have one, visit [API keys](https://platform.openai.com/account/api-keys) on OpenAI's site to create a new key for yourself. Note that API usage charges may apply.

If you encounter errors due to missing modules in your Python environment, install the modules with `pip install`, and then re-run the notebook. For more help, refer to [Installing Python Modules](https://docs.python.org/3/installing/index.html).

## Install the client library

The client library provides Python support for the ValidMind Developer Framework. To install it:

In [63]:
%pip install -q validmind


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Initialize the client library

Every documentation project in the Platform UI comes with a _code snippet_ that lets the client library associate your documentation and tests with the right project on the Platform UI when you run this notebook.

Get your code snippet by creating a documentation project:

1. In a browser, log into the [Platform UI](https://app.prod.validmind.ai).

2. Go to **Documentation Projects** and click **Create new project**.

3. Select **`[Demo] Foundation Model - Text Sentiment Analysis`** and **`Initial Validation`** for the model name and type, give the project a unique  name to make it yours, and then click **Create project**.

4. Go to **Documentation Projects** > **YOUR_UNIQUE_PROJECT_NAME** > **Getting Started** and click **Copy snippet to clipboard**.

Next, replace this placeholder with your own code snippet:

In [64]:
## Replace this placeholder with the code snippet from your own project ##



import validmind as vm

vm.init(
  api_host = "http://localhost:3000/api/v1/tracking",
  api_key = "e6aa0fce77559ce6ba13b8c2de1f2dc5",
  api_secret = "707e4d8e634b3074e1f4a56b0d661b9854abe4892cd3e4f1cdd6a5ae4596d24b",
  project = "clo3ccgsn004j3wy6uyxyvki9"
)
  
  
  
  
  

2023-11-02 10:43:31,707 - INFO(validmind.api_client): Connected to ValidMind. Project: [Demo] Customer Churn Model - Initial Validation (clo3ccgsn004j3wy6uyxyvki9)


## Initialize the Python environment

Next, let's import the necessary libraries and set up your Python environment for data analysis:

In [65]:
import pandas as pd
import xgboost as xgb

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

%matplotlib inline

## Load the sample dataset

In [66]:
# Import the sample dataset from the library

from validmind.datasets.classification import customer_churn as demo_dataset

# You can also try a different dataset with:
# from validmind.datasets.classification import taiwan_credit as demo_dataset

df = demo_dataset.load_data()

In [67]:
demo_dataset.class_labels

{'0': 'Did not exit', '1': 'Exited'}

In [68]:
demo_dataset.target_column

'Exited'

df

### Prepocess the raw dataset

Preprocessing performs a number of operations to get ready for the subsequent steps:

- Preprocess the data: Splits the DataFrame (`df`) into multiple datasets (`train_df`, `validation_df`, and `test_df`) using `demo_dataset.preprocess` to simplify preprocessing.
- Separate features and targets: Drops the target column to create feature sets (`x_train`, `x_val`) and target sets (`y_train`, `y_val`).
- Initialize XGBoost classifier: Creates an `XGBClassifier` object with early stopping rounds set to 10.
- Set evaluation metrics: Specifies metrics for model evaluation as "error," "logloss," and "auc."
- Fit the model: Trains the model on `x_train` and `y_train` using the validation set `(x_val, y_val)`. Verbose output is disabled.

In [69]:
# target_column = demo_dataset.target_column
# df =df[feature_columns + [target_column]]

train_val_df, test_df = train_test_split(df, test_size=0.20)
train_df, validation_df = train_test_split(train_val_df, test_size=0.25)
train_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
27,533,France,Male,36,7,85311.70,1,0,1,156731.91,0
1420,577,France,Female,26,1,180530.51,1,0,0,123454.62,0
5906,647,Spain,Male,33,7,121260.19,2,1,0,77216.48,0
1693,665,France,Male,33,2,101286.11,1,1,1,159840.51,0
5898,596,Spain,Male,20,3,187294.46,1,1,0,103456.47,0
...,...,...,...,...,...,...,...,...,...,...,...
2175,741,Germany,Male,36,8,116993.43,2,1,0,168816.22,0
23,756,Germany,Male,36,2,136815.64,1,1,1,170041.95,0
160,501,France,Male,57,10,0.00,2,1,1,47847.19,0
1958,700,France,Male,38,9,65962.63,1,1,1,100950.48,0


### Initialize the training and test datasets

Before you can run a test suite, which are just a collection of tests, you must first initialize a ValidMind dataset object using the [`init_dataset`](https://docs.validmind.ai/validmind/validmind.html#init_dataset) function from the ValidMind (`vm`) module. 

This function takes a number of arguments: 

- `dataset` — the raw dataset that you want to analyze
- `target_column` — the name of the target column in the dataset 
- `feature_columns` - the names of the feature columns in the dataset 
- `class_labels` — the list of class labels used for classification model training

With the datasets ready, you can now initialize the training and test datasets (`train_df` and `test_df`) created earlier into their own dataset objects using [`vm.init_dataset()`](https://docs.validmind.ai/validmind/validmind.html#init_dataset):

In [70]:
feature_columns = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']

vm_train_ds = vm.init_dataset(
    dataset=train_df,
    target_column=demo_dataset.target_column,
    feature_columns=feature_columns
)

vm_test_ds = vm.init_dataset(
    dataset=test_df,
    target_column=demo_dataset.target_column,
    feature_columns=feature_columns
)

2023-11-02 10:43:31,873 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...
2023-11-02 10:43:31,899 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...


### Defining custom features

This section shows how defining custom features enables us to:

- Use these custom features to compute dataset metrics.
- Employ the columns not included as features for additional purposes, such as segmenting the metrics.

In the following example we use the metric `DatasetDescription` to show how the output changes when customizing features.




1. Running metric with all the features.

In [71]:
test = vm.tests.run_test(
    test_id="validmind.data_validation.DatasetDescription",
    dataset=vm_train_ds
)

2. Running metric with a subset of features.

In [72]:
xtest = vm.tests.run_test(
    test_id="validmind.data_validation.DatasetDescription",
    dataset=vm_train_ds
)